# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function to minimize the percentage of total transaction value lost to fraud. At the end of this demo, we also show you how introducing the right objective during the training is over 4x better than using a generic machine learning metric like AUC.

In [ ]:
import evalml
from evalml import AutoClassificationSearch
from evalml.objectives import FraudCost

## Configure "Cost of Fraud" 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for the cost of fraud. These parameters are

* `retry_percentage` - what percentage of customers will retry a transaction if it is declined?
* `interchange_fee` - how much of each successful transaction do you collect?
* `fraud_payout_percentage` - the percentage of fraud will you be unable to collect
* `amount_col` - the column in the data the represents the transaction amount

Using these parameters, EvalML determines attempt to build a pipeline that will minimize the financial loss due to fraud.

In [ ]:
fraud_objective = FraudCost(retry_percentage=.5,
                            interchange_fee=.02,
                            fraud_payout_percentage=.75,
                            amount_col='amount')

## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [ ]:
X, y = evalml.demos.load_fraud(n_rows=2500)

EvalML natively supports one-hot encoding. Here we keep 1 out of the 6 categorical columns to decrease computation time.

In [ ]:
X = X.drop(['datetime', 'expiration_date', 'country', 'region', 'provider'], axis=1)

X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=0.2, random_state=0)

print(X.dtypes)

Because the fraud labels are binary, we will use `AutoClassificationSearch`. When we call `.search()`, the search for the best pipeline will begin. 

In [ ]:
automl = AutoClassificationSearch(objective=fraud_objective,
                                  additional_objectives=['auc', 'recall', 'precision'],
                                  max_pipelines=5)

automl.search(X_train, y_train)

### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the fraud detection objective we defined

In [ ]:
automl.rankings

to select the best pipeline we can run

In [ ]:
best_pipeline = automl.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [ ]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [ ]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data using both the fraud cost score and the AUC.

In [ ]:
best_pipeline.score(X_holdout, y_holdout, other_objectives=["auc", fraud_objective])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the fraud cost objective to see how the best pipelines compare.

In [ ]:
automl_auc = AutoClassificationSearch(objective='auc',
                                   additional_objectives=['recall', 'precision'],
                                   max_pipelines=5)

automl_auc.search(X_train, y_train)

like before, we can look at the rankings and pick the best pipeline

In [ ]:
automl_auc.rankings

In [ ]:
best_pipeline_auc = automl_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

In [ ]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  other_objectives=["auc", fraud_objective])

In [ ]:
# fraud score on fraud optimized again
best_pipeline.score(X_holdout, y_holdout, other_objectives=["auc", fraud_objective])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for fraud cost. However, the losses due to fraud are over 3% of the total transaction amount when optimized for AUC and under 1% when optimized for fraud cost. As a result, we lose more than 2% of the total transaction amount by not optimizing for fraud cost specifically.

This happens because optimizing for AUC does not take into account the user-specified `retry_percentage`, `interchange_fee`, `fraud_payout_percentage` values. Thus, the best pipelines may produce the highest AUC but may not actually reduce the amount loss due to your specific type fraud.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.